In [71]:
import pyomo.environ as pe
import pyomo.opt as po
import pandas as pd
import math

In [73]:
# Define the predefined assignments from your table
predefined_assignments = {
    '2540': ['2507', '2538', '2541', '2551', '2562', '2571', '2586', '2599', '2603', '2613', '2618', '2631', '2642', '2672', '2673', '2681'],
    '2564': ['2501', '2502', '2503', '2504', '2505', '2506', '2509', '2510', '2511', '2512', '2513', '2514', '2515', '2518', '2519', '2521', '2522', '2523', '2524', '2525', '2526', '2533', '2537', '2539', '2543', '2548', '2550', '2553', '2557', '2561', '2563', '2568', '2573', '2576', '2578', '2581', '2590', '2593', '2596', '2597', '2600', '2601', '2604', '2605', '2606', '2607', '2616', '2617', '2619', '2629', '2630', '2633', '2634', '2636', '2640', '2641', '2647', '2658', '2660', '2661', '2664', '2666', '2668', '2669', '2674', '2675', '2676', '2680', '2683', '2686', '2689'],
    '2565': ['2516', '2517', '2542', '2547', '2552', '2555', '2559', '2572', '2584', '2594', '2595', '2608', '2609', '2612', '2637', '2639', '2649', '2653', '2654', '2678', '2687'],
    '2583': ['2520', '2530', '2546', '2560', '2570', '2602', '2610', '2611', '2614', '2615', '2620', '2621', '2623', '2626', '2635', '2638', '2646', '2651', '2656', '2663', '2665', '2671', '2679', '2684', '2688'],
    '2644': ['2527', '2531', '2549', '2554', '2556', '2558', '2566', '2579', '2580', '2585', '2643', '2652', '2655', '2659', '2667', '2670'],
    '2645': ['2532', '2535', '2569', '2577', '2582', '2587', '2588', '2591', '2592', '2598', '2622', '2624', '2625', '2627', '2628', '2648', '2650', '2657', '2662', '2677', '2682', '2685']
}

# Create the arc set A = {(i,j) : store i is assigned to outlet j}
arc_list = []
for outlet, stores in predefined_assignments.items():
    for store in stores:
        arc_list.append((store, outlet))


In [75]:

df_stores = pd.read_excel("Python Data Full Scale.xlsx", sheet_name="Stores", usecols=["Store Number"])
store_list = df_stores["Store Number"].dropna().astype(str).tolist()

outlet_list = ["2540", "2564", "2565", "2583", "2644", "2645"]

df_sku = pd.read_excel("Python Data Full Scale.xlsx", sheet_name="SKU List", usecols=["SKU"])
sku_list = df_sku["SKU"].dropna().astype(str).tolist()

# Filter arc_list to only include stores that exist in your data
arc_list = [(i, j) for (i, j) in arc_list if i in store_list]

print(f"Number of valid arcs: {len(arc_list)}")

Number of valid arcs: 171


In [77]:
# Model with arc-based formulation
model = pe.ConcreteModel()

# Sets
model.S = pe.Set(initialize=store_list)    # Retail stores (i)
model.O = pe.Set(initialize=outlet_list)   # Outlet stores (j)
model.K = pe.Set(initialize=sku_list)      # SKUs (k)
model.A = pe.Set(initialize=arc_list)      # Allowed arcs (i,j)

In [79]:
# Stock

df = pd.read_excel("Python Data Full Scale.xlsx", sheet_name="Stock")
stock_dict = {(str(row['Store']), str(row['SKU'])): int(row['Stock']) for idx, row in df.iterrows()}
model.Stock = pe.Param(model.S, model.K, initialize=stock_dict, within=pe.NonNegativeIntegers, default=0)

print("Number of (store, SKU) pairs:", len(stock_dict))


Number of (store, SKU) pairs: 12997


In [81]:
#Capacity

df_cap = pd.read_excel("Python Data Full Scale.xlsx", sheet_name="Outlet Capacity")
cap_dict = {str(row['Store']): int(row['February Capacity (Delisted)']) for idx, row in df_cap.iterrows()}
model.Cap = pe.Param(model.O, initialize=cap_dict)

print("Number of outlets in cap_dict:", len(cap_dict))
print("cap_dict:", cap_dict)

Number of outlets in cap_dict: 6
cap_dict: {'2540': 30376, '2564': 24704, '2565': 27918, '2583': 20768, '2644': 25768, '2645': 15526}


In [107]:
# Pallet Size

df_psize = pd.read_excel("Python Data Full Scale.xlsx", sheet_name="Pallet")
psize_dict = {str(row['articlecode']): int(row['PalletQuantiyPieces']) for idx, row in df_psize.iterrows()}
model.PSize = pe.Param(model.K, initialize=psize_dict)



'pyomo.core.base.param.IndexedParam'>) on block unknown with a new Component
(type=<class 'pyomo.core.base.param.IndexedParam'>). This is usually
indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().


In [87]:
# Revenue

df_rev = pd.read_excel("Python Data Full Scale.xlsx", sheet_name="Revenue")
revenue_dict = {str(int(row['articlecode'])): float(row['revenue']) for _, row in df_rev.iterrows()}
model.r = pe.Param(model.K, initialize=revenue_dict)


In [89]:
model.c_p = pe.Param(initialize=175)  # €/pallet
model.f = pe.Param(initialize=60)     # €/truck
model.M = pe.Param(initialize=33)     # Big M


In [91]:
# Decision variables (modified for arcs)
model.x = pe.Var(model.A, model.K, domain=pe.NonNegativeIntegers)  # Only for allowed arcs
model.p = pe.Var(model.A, domain=pe.NonNegativeReals)              # Only for allowed arcs
model.w = pe.Var(model.A, domain=pe.Binary)                        # Only for allowed arcs
model.s = pe.Var(model.S, model.K, domain=pe.Binary)


In [93]:
# Objective: maximize net profit
revenue_expr = sum(model.r[k] * model.x[i, j, k] for (i, j) in model.A for k in model.K)
cost_expr = sum(model.c_p * model.p[i, j] + model.f * model.w[i, j] for (i, j) in model.A)
model.obj = pe.Objective(expr=revenue_expr - cost_expr, sense=pe.maximize)

In [95]:
# (1) Supply constraint: ship all OR none of each SKU from each store
model.SupplyConstraint = pe.ConstraintList()
for i in model.S:
    for k in model.K:
        # Sum over all outlets j where arc (i,j) exists
        outlets_for_store_i = [j for (store, j) in model.A if store == i]
        if outlets_for_store_i:  # Only add constraint if store i has assigned outlets
            expr = sum(model.x[i, j, k] for j in outlets_for_store_i) == model.Stock[i, k] * model.s[i, k]
            model.SupplyConstraint.add(expr)

In [96]:
# (2) Outlet capacity constraint
model.OutletCapacityConstraint = pe.ConstraintList()
for j in model.O:
    # Sum over all stores i where arc (i,j) exists
    stores_for_outlet_j = [i for (i, outlet) in model.A if outlet == j]
    if stores_for_outlet_j:  # Only add constraint if outlet j has assigned stores
        expr = sum(model.x[i, j, k] for i in stores_for_outlet_j for k in model.K)
        model.OutletCapacityConstraint.add(expr <= model.Cap[j])

In [98]:
# (3) Truck capacity constraint
model.TruckCapacityConstraint = pe.ConstraintList()
for (i, j) in model.A:
    expr = model.p[i, j] <= model.M * model.w[i, j]
    model.TruckCapacityConstraint.add(expr)

In [101]:
# (4) Pallet-mixing constraint
model.PalletMixingConstraint = pe.ConstraintList()
for (i, j) in model.A:
    expr = sum(model.x[i, j, k] / model.PSize[k] for k in model.K) == model.p[i, j]
    model.PalletMixingConstraint.add(expr)


In [103]:
# Solve the model
solver = po.SolverFactory('gurobi')
solver.options['TimeLimit'] = 180
result = solver.solve(model, tee=True)

print(result.solver.status)
print(result.solver.termination_condition)
print("Objective value = " + str(pe.value(model.obj)))


Set parameter Username
Set parameter LicenseID to value 2617608
Academic license - for non-commercial use only - expires 2026-02-03
Read LP format model from file C:\Users\User\AppData\Local\Temp\tmpr4s9w_o6.pyomo.lp
Reading time = 0.39 seconds
x1: 145356 rows, 158347 columns, 448534 nonzeros
Set parameter TimeLimit to value 180
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Non-default parameters:
TimeLimit  180

Optimize a model with 145356 rows, 158347 columns and 448534 nonzeros
Model fingerprint: 0x3f13d58a
Variable types: 171 continuous, 158176 integer (13168 binary)
Coefficient statistics:
  Matrix range     [4e-05, 9e+02]
  Objective range  [2e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+04, 3e+04]
Found heuristic solution: objective -0.0000000
Presolve removed 

In [109]:
# Calculate rounded cost and objective
if result.solver.termination_condition == pe.TerminationCondition.optimal:
    revenue_value = sum(model.r[k] * pe.value(model.x[i, j, k]) for (i, j) in model.A for k in model.K)
    rounded_cost = sum(model.c_p * math.ceil(pe.value(model.p[i, j])) + model.f * pe.value(model.w[i, j]) for (i, j) in model.A)
    rounded_obj_value = revenue_value - rounded_cost
    print(f"Objective value with rounded pallets: {rounded_obj_value:.2f}")

Objective value with rounded pallets: 1594719.35


In [119]:
# Print solution summary with rounded pallets
print("\nAssignment Summary:")
for (i, j) in model.A:
    total_pallets = pe.value(model.p[i, j])
    if total_pallets > 0.001:  # Only show active routes
        rounded_pallets = math.ceil(total_pallets)
        print(f"Store {i} → Outlet {j}: {rounded_pallets} pallets (actual: {total_pallets:.2f})")


Assignment Summary:
Store 2507 → Outlet 2540: 2 pallets (actual: 1.91)
Store 2538 → Outlet 2540: 1 pallets (actual: 0.84)
Store 2541 → Outlet 2540: 2 pallets (actual: 1.39)
Store 2551 → Outlet 2540: 1 pallets (actual: 0.95)
Store 2562 → Outlet 2540: 2 pallets (actual: 1.14)
Store 2571 → Outlet 2540: 2 pallets (actual: 1.37)
Store 2586 → Outlet 2540: 3 pallets (actual: 2.06)
Store 2599 → Outlet 2540: 2 pallets (actual: 1.21)
Store 2603 → Outlet 2540: 2 pallets (actual: 1.57)
Store 2613 → Outlet 2540: 2 pallets (actual: 1.81)
Store 2618 → Outlet 2540: 2 pallets (actual: 1.07)
Store 2631 → Outlet 2540: 1 pallets (actual: 0.95)
Store 2642 → Outlet 2540: 2 pallets (actual: 1.96)
Store 2672 → Outlet 2540: 3 pallets (actual: 2.37)
Store 2673 → Outlet 2540: 1 pallets (actual: 0.82)
Store 2681 → Outlet 2540: 2 pallets (actual: 1.37)
Store 2502 → Outlet 2564: 1 pallets (actual: 0.76)
Store 2503 → Outlet 2564: 1 pallets (actual: 0.97)
Store 2504 → Outlet 2564: 1 pallets (actual: 0.53)
Store 2505

In [117]:
# -------------------------------------------------
# Build table: Store | Outlet | SKU | Shipped
# -------------------------------------------------
rows = []
for (i, j) in model.A:  # Only iterate over valid arcs
    for k in model.K:
        qty = pe.value(model.x[i, j, k]) or 0
        if qty > 1e-6:        # keep only positive flows
            rows.append({"Store": i, "Outlet": j, "SKU": k, "Shipped": qty})

df_ship = pd.DataFrame(rows)

# -------------------------------------------------
# Export to Excel
# -------------------------------------------------
file_path = "arc_assignment_summary_single.xlsx"
df_ship.to_excel(file_path, index=False)
print(f"Shipment data exported to {file_path}")
print(f"Total shipment records: {len(df_ship)}")

Shipment data exported to arc_assignment_summary_single.xlsx
Total shipment records: 11296


In [53]:
# ----------------------------------------------------------
# Build summary table: Store | SKU | Shipped | Stock | Over-shipped
# ----------------------------------------------------------
rows = []
for (i, k), stock_val in stock_dict.items():            # only pairs that have stock data
    # Sum shipped quantities only over outlets that store i is assigned to
    outlets_for_store_i = [j for (store, j) in model.A if store == i]
    shipped_val = sum(pe.value(model.x[i, j, k]) for j in outlets_for_store_i if (i, j) in model.A)
    
    rows.append({
        "Store":        i,
        "SKU":          k,
        "Shipped":      shipped_val,
        "Stock":        stock_val,
        "Over-shipped": shipped_val > stock_val
    })

df_summary = pd.DataFrame(rows)

# ----------------------------------------------------------
# Export to Excel   (second report)
# ----------------------------------------------------------
file_path_summary = "shipped_vs_stock_arc_based_single.xlsx"             
df_summary.to_excel(file_path_summary, index=False)
print(f"Stock comparison data exported to {file_path_summary}")
print(f"Total store-SKU records: {len(df_summary)}")

# ----------------------------------------------------------
# Check for over-shipments
# ----------------------------------------------------------
over_shipped = df_summary[df_summary['Over-shipped'] == True]
if len(over_shipped) > 0:
    print(f"WARNING: {len(over_shipped)} store-SKU combinations are over-shipped!")
else:
    print("✓ No over-shipments detected")

Stock comparison data exported to shipped_vs_stock_arc_based_single.xlsx
Total store-SKU records: 12997
✓ No over-shipments detected


In [55]:
# Print number of pallets shipped per route (rounded up)
for (i, j) in model.A:  # Only iterate over valid arcs
    pallets = pe.value(model.p[i, j])
    if pallets > 1e-6:
        rounded_pallets = math.ceil(pallets)
        print(f"p[{i},{j}] = {rounded_pallets}")

# Create DataFrame with rounded up pallet values
rows = []
for (i, j) in model.A:  # Only iterate over valid arcs
    pallets = pe.value(model.p[i, j]) or 0
    if pallets > 1e-6:                    # keep only routes that ship pallets
        rounded_pallets = math.ceil(pallets)
        rows.append({"Store": i,
                     "Outlet": j,
                     "Pallets": rounded_pallets})

df_pallets = pd.DataFrame(rows)

# -------------------------------------------------
# Export to Excel
# -------------------------------------------------
file_path_pallets = "pallets_by_route_arc_based_single.xlsx"
df_pallets.to_excel(file_path_pallets, index=False)
print(f"Pallet route data exported to {file_path_pallets}")
print(f"Total active routes: {len(df_pallets)}")

# -------------------------------------------------
# Summary statistics
# -------------------------------------------------
if len(df_pallets) > 0:
    total_pallets = df_pallets['Pallets'].sum()
    avg_pallets_per_route = df_pallets['Pallets'].mean()
    print(f"Total pallets shipped: {total_pallets}")
    print(f"Average pallets per route: {avg_pallets_per_route:.2f}")
else:
    print("No active routes found")

p[2507,2540] = 2
p[2538,2540] = 1
p[2541,2540] = 2
p[2551,2540] = 1
p[2562,2540] = 2
p[2571,2540] = 2
p[2586,2540] = 3
p[2599,2540] = 2
p[2603,2540] = 2
p[2613,2540] = 2
p[2618,2540] = 2
p[2631,2540] = 1
p[2642,2540] = 2
p[2672,2540] = 3
p[2673,2540] = 1
p[2681,2540] = 2
p[2502,2564] = 1
p[2503,2564] = 1
p[2504,2564] = 1
p[2505,2564] = 1
p[2506,2564] = 1
p[2509,2564] = 3
p[2510,2564] = 1
p[2511,2564] = 1
p[2512,2564] = 1
p[2513,2564] = 1
p[2514,2564] = 1
p[2515,2564] = 1
p[2518,2564] = 1
p[2519,2564] = 1
p[2521,2564] = 1
p[2522,2564] = 2
p[2523,2564] = 1
p[2524,2564] = 2
p[2525,2564] = 2
p[2526,2564] = 1
p[2533,2564] = 1
p[2537,2564] = 2
p[2539,2564] = 1
p[2543,2564] = 1
p[2548,2564] = 1
p[2550,2564] = 1
p[2557,2564] = 1
p[2561,2564] = 1
p[2563,2564] = 2
p[2568,2564] = 1
p[2573,2564] = 1
p[2578,2564] = 1
p[2581,2564] = 1
p[2590,2564] = 1
p[2593,2564] = 2
p[2596,2564] = 1
p[2597,2564] = 2
p[2600,2564] = 3
p[2601,2564] = 1
p[2604,2564] = 1
p[2605,2564] = 2
p[2606,2564] = 1
p[2607,2564] =

In [115]:
# Calculate total rounded up pallets
total_pallets_rounded = sum(math.ceil(pe.value(model.p[i, j])) for i in model.S for j in model.O)

# 1) variable cost with rounded up pallets (€/pallet)
var_cost_rounded = model.c_p * total_pallets_rounded

# 2) fixed cost (€/truck dispatched) - this stays the same
fixed_cost_rounded = model.f * sum(
    pe.value(model.w[i, j])                # 1 if truck used on route i→j
    for i in model.S for j in model.O
)

# 3) total cost with rounded pallets
total_cost_rounded = var_cost_rounded + fixed_cost_rounded

print(f"Total pallets shipped (rounded up)                    : {total_pallets_rounded:,}")
print(f"Variable transport cost with rounded pallets (€/pallet): {var_cost_rounded:,.2f}")
print(f"Fixed truck cost (€/truck)                             : {fixed_cost_rounded:,.2f}")
print(f"------------------------------------------------------------------")
print(f"Total transport cost with rounded pallets              : {total_cost_rounded:,.2f}")

KeyError: "Index '('2623', '2540')' is not valid for indexed component 'p'"